In [1]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm

# from timelab import MultiColumn, from_data, from_xy_data
# from timelab.utils import smash_array
import sys
sys.path.append('../')
from wavy import *

In [2]:
df = pd.read_pickle('processed.pkl')

In [3]:
# df = df[['LNC', 'MAS', 'CSX']]
df = df[['MAS']]

In [4]:
df.head(5)

MAS                                            
                 Open       High        Low      Close     Volume
Date                                                             
2005-12-21  18.991695  19.154659  18.903944  19.085712  1384200.0
2005-12-22  19.085703  19.298810  18.910202  18.929007  1576900.0
2005-12-23  18.954090  18.954090  18.684571  18.816196  2090500.0
2005-12-27  18.929008  18.991687  18.803650  18.847525  1609000.0
2005-12-28  18.853798  18.935281  18.747245  18.803656  2026300.0

In [5]:
panel = from_data(df, lookback=10, horizon=1)

In [6]:
a = panel.x.pct_change()

/Users/ibiscp/GitHub/wavy/notebooks/../wavy/block.py:882: FutureWarning: In a future version of pandas all arguments of DataFrame.fillna except for the argument 'value' will be keyword-only
  return super().fillna(value, method, axis, inplace, limit, downcast)


In [7]:
a[0]

MAS                                        
                Open      High       Low     Close    Volume
Date                                                        
2005-12-21       NaN       NaN       NaN       NaN       NaN
2005-12-22  0.004950  0.007526  0.000331 -0.008211  0.139214
2005-12-23 -0.006896 -0.017862 -0.011932 -0.005960  0.325702
2005-12-27 -0.001323  0.001984  0.006373  0.001665 -0.230328
2005-12-28 -0.003973 -0.002970 -0.003000 -0.002328  0.259354
2005-12-29 -0.002659 -0.000331 -0.001337 -0.000667 -0.240734
2005-12-30 -0.000667  0.002649  0.003348  0.007004  0.086383
2006-01-03  0.020014  0.030714  0.007675  0.033787  1.172430
2006-01-04  0.017640 -0.000968  0.016781 -0.009029 -0.343349
2006-01-05 -0.007115  0.000323  0.000655 -0.004556  0.082456

In [10]:
panel.x = panel.x.iloc[1:]

In [11]:
panel

size                            3737
lookback                           9
horizon                            1
num_xassets                        1
num_yassets                        1
num_xchannels                      5
num_ychannels                      5
start            2005-12-22 00:00:00
end              2020-11-10 00:00:00
Name: TimePanel, dtype: object


<TimePanel, size 3737>

In [ ]:
type(panel.x)

In [ ]:
panel.x = panel.x.shift(2)

In [ ]:
panel.x.first

In [ ]:
panel.x.matrix[0]

In [ ]:
type(panel.x[0])

In [ ]:
a = getattr(TimeBlock, 'iloc')

In [ ]:
a

In [ ]:
a = panel.x.iloc[:2]

In [ ]:
a.first

In [ ]:
getattr(panel.x.first, a[0].__name__)

In [ ]:
panel.x.first.iloc[1:]

In [16]:
panel.x = panel.x.fillna(0)
# WHAT? I commented this. Please take a look at panel.py >> 197

100%|██████████| 3737/3737 [00:00<00:00, 4915.71it/s]


In [17]:
panel.x = panel.x.fillna(0)

100%|██████████| 3737/3737 [00:00<00:00, 7198.63it/s]


In [18]:
panel.x[0]

MAS                                            
                 Open       High        Low      Close     Volume
Date                                                             
2005-12-22  19.085703  19.298810  18.910202  18.929007  1576900.0
2005-12-23  18.954090  18.954090  18.684571  18.816196  2090500.0
2005-12-27  18.929008  18.991687  18.803650  18.847525  1609000.0
2005-12-28  18.853798  18.935281  18.747245  18.803656  2026300.0
2005-12-29  18.803659  18.929017  18.722176  18.791122  1538500.0
2005-12-30  18.791119  18.979156  18.784852  18.922745  1671400.0
2006-01-03  19.167201  19.562078  18.929022  19.562078  3631000.0
2006-01-04  19.505303  19.543152  19.246662  19.385445  2384300.0
2006-01-05  19.366519  19.549459  19.259278  19.297127  2580900.0

In [19]:
panel.y[0]

MAS                                          
                 Open     High        Low      Close     Volume
Date                                                           
2006-01-06  19.398057  19.4296  18.924935  19.177267  2672700.0

In [20]:
panel.y = panel.y.filter(channels='High')
# Too slow. Just imagine an initial df of 1M rows.
# Let's time each of the panel/side functions and see try to speed up the slow ones.

100%|██████████| 3737/3737 [00:45<00:00, 81.43it/s] 


In [ ]:
panel.y[0]

In [24]:
panel.y.values
# ???

<function wavy.side.PanelSide.__getattr__.<locals>.wrapper(*args, **kwargs)>

In [25]:
baseline = Baseline(panel)

In [28]:
y_true = panel.test.y.flatten()
y_pred = baseline.predict()

In [36]:
y_pred = y_pred.predicted

In [31]:
y_true.shape

(373,)

In [38]:
y_true[:3]

array([37.50564372, 36.82783211, 36.65101656])

In [37]:
y_pred[:3]

Date
2019-05-22          NaN
2019-05-23    37.505644
2019-05-24    36.827832
Name: predicted, dtype: float64

In [ ]:
# Maybe the predictions should come as series/dataframes indeed. Here, the INDEX is really important, and we could remove the first NaN as long as we keep the index of the second value explicit.

# Let's think of a way to make indexes intuitive. They are simple when input index is dates, but when this is not passed, just numeric indexes would work?

# This means quite a shift in the way we treat a panel's/side index. It should be fixed in a way that even when dropping samples or adding steps, etc. the index is still consistent.

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_true, y_pred)
# NaN makes it not computable.

In [ ]:
mean_squared_error(y_true[1:], y_pred[1:]) # horizon = 1, thus that's the number of nans generated with baseline model.

# If the index is explicit and we use dataframes, we could hide NaN values...
# metrics such as mean_squared_error could be part of the Model, e.g. model.validate('mse'), model.test('mse)